# Import libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,StandardScaler

In [2]:
import warnings

# Suppress all FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Load the Datasets
We load all the required datasets.
For columns that represent dates (e.g., date), we use parse_dates=["date"] when reading the files to automatically convert them into datetime64 objects. This makes time-based operations much easier more convenient for time series analysis.

In [3]:
train = pd.read_csv("train.csv", parse_dates=["date"])
test = pd.read_csv("test.csv", parse_dates=["date"])
stores = pd.read_csv("stores.csv")
transactions = pd.read_csv("transactions.csv", parse_dates=["date"])
holidays = pd.read_csv("holidays_events.csv", parse_dates=["date"])
oil = pd.read_csv("oil.csv", parse_dates=["date"])

In [4]:
train.columns

Index(['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion'], dtype='object')

In [5]:
test.columns

Index(['id', 'date', 'store_nbr', 'family', 'onpromotion'], dtype='object')

In [6]:
stores.columns

Index(['store_nbr', 'city', 'state', 'type', 'cluster'], dtype='object')

In [7]:
transactions.columns

Index(['date', 'store_nbr', 'transactions'], dtype='object')

In [8]:
holidays.columns

Index(['date', 'type', 'locale', 'locale_name', 'description', 'transferred'], dtype='object')

In [9]:
oil.columns

Index(['date', 'dcoilwtico'], dtype='object')

# Merge Auxiliary Data into Train

- **stores.csv** → Adds store information (city, state, type, cluster).

- **transactions.csv** → Adds daily transactions per store.

- **oil.csv** → Adds oil prices (missing values forward-filled).

- **holidays_events.csv** → Adds holiday/event type for each date.

In [10]:
train = train.merge(stores, on="store_nbr", how="left")
train = train.merge(transactions, on=["date", "store_nbr"], how="left")
oil["dcoilwtico"] = oil["dcoilwtico"].fillna(method="ffill")
train = train.merge(oil, on="date", how="left")
train = train.merge(holidays[["date","type"]], on="date", how="left")
train.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,transactions,dcoilwtico,type_y
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday


In [11]:
train.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,transactions,dcoilwtico,type_y
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday


### Renaming Columns for Clarity

- After merging multiple datasets, some columns inherited generic names like `type_x` and `type_y`.  
- To make the dataset more interpretable, we renamed them:

    - **type_x** (coming from stores.csv) →  **store_type**

        Indicates the type of store (A, B, C, or D).

    - **type_y** (coming from holidays_events.csv) → **day_type**

        Classifies each date as either a special event (Holiday, Transfer, Additional, Event, etc.) or a regular Work Day (for dates without special events).

In [12]:
train = train.rename(columns={
    'type_x': 'store_type',
    'type_y': 'day_type'
})

## Set_index ID Column

In [13]:
train.set_index("id", inplace=True)

# Merge Auxiliary Data into Test

- **stores.csv** → Adds store-level information (`city`, `state`, `type`, `cluster`).  

- **transactions.csv** → Adds the number of daily transactions for each store (based on `date` + `store_nbr`).  

- **oil.csv** → Adds oil price (`dcoilwtico`) for each date.  

- **holidays_events.csv** → Adds holiday/event type (`type`) for each date.  

**Note**: Unlike the train dataset, the test dataset does not include the `sales` column, because `sales` is the target variable we aim to predict. All other auxiliary data is merged to provide the necessary context for forecasting.  


In [14]:
test = test.merge(stores, on="store_nbr", how="left")
test = test.merge(transactions, on=["date", "store_nbr"], how="left")
test = test.merge(oil, on="date", how="left")
test = test.merge(holidays[["date","type"]], on="date", how="left")
test.head()

,id,date,store_nbr,family,onpromotion,city,state,type_x,cluster,transactions,dcoilwtico,type_y
0,3000888,2017-08-16,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,NaN,46.8,NaN
1,3000889,2017-08-16,1,BABY CARE,0,Quito,Pichincha,D,13,NaN,46.8,NaN
2,3000890,2017-08-16,1,BEAUTY,2,Quito,Pichincha,D,13,NaN,46.8,NaN
3,3000891,2017-08-16,1,BEVERAGES,20,Quito,Pichincha,D,13,NaN,46.8,NaN
4,3000892,2017-08-16,1,BOOKS,0,Quito,Pichincha,D,13,NaN,46.8,NaN


### Renaming Columns for Clarity (Test Set)

- Similar to the train dataset, merging multiple files created generic column names (`type_x`, `type_y`).  
- To make the test dataset more interpretable, we renamed them:

    - **type_x** (from `stores.csv`) → **store_type**  
      Indicates the type of store (A, B, C, or D).  

    - **type_y** (from `holidays_events.csv`) → **day_type**  
      Classifies each date as either a special event (Holiday, Transfer, Additional, Event, etc.) or a regular Work Day (for dates without special events).  


In [15]:
test = test.rename(columns={
    'type_x': 'store_type',
    'type_y': 'day_type'
})

## Set_index ID Column

In [16]:
test.set_index("id",inplace=True)

# **Dataset Overview**

## Shape of the dataset

In [17]:
train.shape

(3054348, 12)

In [18]:
train.head()

,date,store_nbr,family,sales,onpromotion,city,state,store_type,cluster,transactions,dcoilwtico,day_type
id,,,,,,,,,,,,
0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday
1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday
2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday
3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday
4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday


## Dataset info (data types + missing values)

In [19]:
train.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 3054348 entries, 0 to 3000887
Data columns (total 12 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   date          3054348 non-null  datetime64[ns]
 1   store_nbr     3054348 non-null  int64         
 2   family        3054348 non-null  object        
 3   sales         3054348 non-null  float64       
 4   onpromotion   3054348 non-null  int64         
 5   city          3054348 non-null  object        
 6   state         3054348 non-null  object        
 7   store_type    3054348 non-null  object        
 8   cluster       3054348 non-null  int64         
 9   transactions  2805231 non-null  float64       
 10  dcoilwtico    2174040 non-null  float64       
 11  day_type      502524 non-null   object        
dtypes: datetime64[ns](1), float64(3), int64(3), object(5)
memory usage: 302.9+ MB


## Missing values count

In [20]:
train.isnull().sum()

date                  0
store_nbr             0
family                0
sales                 0
onpromotion           0
city                  0
state                 0
store_type            0
cluster               0
transactions     249117
dcoilwtico       880308
day_type        2551824
dtype: int64

**Columns with Missing Values**  
   - **`transactions` → 249,117 missing (~8.2%)**  
     - Not all stores have transaction logs available.  
     - Missing values can be imputed using median per store or interpolation.  

   - **`dcoilwtico` → 880,308 missing (~28.8%)**  
     - Oil prices are not recorded on weekends and holidays (markets closed).  
     - We can fix it using forward-fill (`ffill`) or interpolation.  

   - **`day_type` → 2,551,824 missing (~83.5%)**  
     - Represents holiday/event types.  
     - Missing values simply mean “regular work day.”  
     - Can safely replace NaN with `"Work Day"`.  

#### Key Takeaways
- Core dataset is very clean (no missing values in essential features like `sales`, `date`, `store_nbr`).  
- Missing values are expected in auxiliary features (transactions, oil prices, holidays).  

In [21]:
test.isna().sum()

date                0
store_nbr           0
family              0
onpromotion         0
city                0
state               0
store_type          0
cluster             0
transactions    28512
dcoilwtico       7128
day_type        26730
dtype: int64

## **Columns with Missing Values — Test Dataset**

- **`transactions` → 28,512 missing (100%)**  
  - All transaction values are missing in the test set.  
  - Since this feature is not available for prediction, we will drop the `transactions` column from both **train** and **test** to keep the datasets aligned.  

- **`dcoilwtico` → 7,128 missing (~2.4%)**  
  - Oil prices are not recorded on weekends and holidays (markets closed).  
  - We can handle missing values using **forward-fill (`ffill`)**.  

- **`day_type` → 26,730 missing (~9.0%)**  
  - Represents holiday/event types.  
  - Missing values simply mean “regular work day.”  
  - We can safely replace NaN with `"Work Day"`.  


## Check duplicates

In [22]:
train.duplicated().sum()

np.int64(30294)

In [23]:
test.duplicated().sum()

np.int64(0)

In [24]:
initial_rows = train.shape[0]
train = train.drop_duplicates()
print(f"Removed {initial_rows - train.shape[0]} duplicate rows.")
train.shape

Removed 30294 duplicate rows.


(3024054, 12)

**We removed all duplicate rows to ensure data consistency:**  

## Check Hidden Duplicated

In [25]:
# Edit BY Null values in day_type

In [26]:
dup = train.drop("day_type",axis=1).duplicated()
dup_index = dup[dup].index
train = train.drop(dup_index)
# edit

## Data Validation & Cleaning Checks

## Handle missing values
- Logical imputation strategies:  
  - `transactions`: median per store.  
  - `dcoilwtico`: forward-fill.  
  - `day_type`: fill NaN with `"Work Day"`.  

**Fill missing values of dcoilwtico column with forward-fill then back-fill for any leading NaNs**

In [27]:
train['dcoilwtico'] = train['dcoilwtico'].fillna(method='ffill').fillna(method='bfill')

**Fill missing values day_type column with "Work Day"**

In [28]:
train['day_type'] = train['day_type'].fillna('Work Day')

In [29]:
train.isnull().sum()


date                 0
store_nbr            0
family               0
sales                0
onpromotion          0
city                 0
state                0
store_type           0
cluster              0
transactions    244398
dcoilwtico           0
day_type             0
dtype: int64

**Fill missing values of dcoilwtico column in the test dataset with forward-fill then back-fill for any leading NaNs**

In [30]:
test['dcoilwtico'] = test['dcoilwtico'].fillna(method='ffill').fillna(method='bfill')

**Fill missing values day_type column in the test dataset with "Work Day"**

In [31]:
test['day_type'] = test['day_type'].fillna('Work Day')

## **Dropping the `transactions` Column**

To keep the datasets consistent, we drop `transactions` from both **train** and **test**:

In [32]:
train.drop("transactions",axis=1,inplace=True)

In [33]:
test.drop("transactions",axis=1,inplace=True)

In [34]:
train.shape

(2977722, 11)

## **Address data inconsistencies**

## Check data types

In [35]:
train.dtypes

date           datetime64[ns]
store_nbr               int64
family                 object
sales                 float64
onpromotion             int64
city                   object
state                  object
store_type             object
cluster                 int64
dcoilwtico            float64
day_type               object
dtype: object

- **Categorical Columns:** family, city, state, store_type, day_type.  
- **Numeric Columns:** id, store_nbr, sales, onpromotion, cluster, transactions, dcoilwtico.  
- **Datetime Columns:** date.


## List of all columns

In [36]:
train.columns

Index(['date', 'store_nbr', 'family', 'sales', 'onpromotion', 'city', 'state',
       'store_type', 'cluster', 'dcoilwtico', 'day_type'],
      dtype='object')

## Unique values exploration

**View all distinct product families available in the dataset.**

In [37]:
train.family.unique()

array(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD'], dtype=object)

**Get unique store types in the dataset**

In [38]:
train.store_type.unique()

array(['D', 'C', 'B', 'E', 'A'], dtype=object)

**Get unique day types in the dataset**

In [39]:
train.day_type.unique()

array(['Holiday', 'Work Day', 'Additional', 'Transfer', 'Event', 'Bridge'],
      dtype=object)

## Count unique values in each column

In [40]:
train.nunique()

date             1671
store_nbr          54
family             33
sales          376834
onpromotion       351
city               22
state              16
store_type          5
cluster            17
dcoilwtico        987
day_type            6
dtype: int64

## Descriptive statistics for all columns

In [41]:
train.describe(include='all')

,date,store_nbr,family,sales,onpromotion,city,state,store_type,cluster,dcoilwtico,day_type
count,2977722,2.977722e+06,2977722,2.977722e+06,2.977722e+06,2977722,2977722,2977722,2.977722e+06,2.977722e+06,2977722
unique,NaN,NaN,33,NaN,NaN,22,16,5,NaN,NaN,6
top,NaN,NaN,AUTOMOTIVE,NaN,NaN,Quito,Pichincha,D,NaN,NaN,Work Day
freq,NaN,NaN,90234,NaN,NaN,992574,1047717,992574,NaN,NaN,2558952
mean,2015-04-23 09:07:13.034111488,2.750000e+01,NaN,3.567573e+02,2.594631e+00,NaN,NaN,NaN,8.481481e+00,6.799561e+01,NaN
min,2013-01-01 00:00:00,1.000000e+00,NaN,0.000000e+00,0.000000e+00,NaN,NaN,NaN,1.000000e+00,2.619000e+01,NaN
25%,2014-02-25 00:00:00,1.400000e+01,NaN,0.000000e+00,0.000000e+00,NaN,NaN,NaN,4.000000e+00,4.641000e+01,NaN
50%,2015-04-23 00:00:00,2.750000e+01,NaN,1.100000e+01,0.000000e+00,NaN,NaN,NaN,8.500000e+00,5.341000e+01,NaN
75%,2016-06-20 00:00:00,4.100000e+01,NaN,1.950000e+02,0.000000e+00,NaN,NaN,NaN,1.300000e+01,9.572000e+01,NaN
max,2017-08-15 00:00:00,5.400000e+01,NaN,1.247170e+05,7.410000e+02,NaN,NaN,NaN,1.700000e+01,1.106200e+02,NaN


## Add Feature Engineering

To enhance our dataset with time-related insights, we extract several features from the `date` column.  
This helps capture seasonal patterns, monthly trends, and weekday effects in sales forecasting.

In [42]:
train["Year"]=pd.to_datetime(train["date"]).dt.year
train["Month"]=pd.to_datetime(train["date"]).dt.month
train["Day"]=pd.to_datetime(train["date"]).dt.day
train["Week_day"]=train["date"].dt.strftime("%A")

In [43]:
test["Year"]=pd.to_datetime(test["date"]).dt.year
test["Month"]=pd.to_datetime(test["date"]).dt.month
test["Day"]=pd.to_datetime(test["date"]).dt.day
test["Week_day"]=test["date"].dt.strftime("%A")

In [44]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2977722 entries, 0 to 3000887
Data columns (total 15 columns):
 #   Column       Dtype         
---  ------       -----         
 0   date         datetime64[ns]
 1   store_nbr    int64         
 2   family       object        
 3   sales        float64       
 4   onpromotion  int64         
 5   city         object        
 6   state        object        
 7   store_type   object        
 8   cluster      int64         
 9   dcoilwtico   float64       
 10  day_type     object        
 11  Year         int32         
 12  Month        int32         
 13  Day          int32         
 14  Week_day     object        
dtypes: datetime64[ns](1), float64(2), int32(3), int64(3), object(6)
memory usage: 329.4+ MB


In [45]:
# Define lag periods (e.g., 7, 14, 28 days)
lag_days = [7, 14, 28]

for lag in lag_days:
    train[f'sales_lag_{lag}'] = train.groupby(['store_nbr','family'])['sales'].shift(lag)

# Create rolling mean features (e.g., 7-day, 14-day, 28-day)
window_sizes = [7, 14, 28]

for window in window_sizes:
    train[f'sales_roll_mean_{window}'] = train.groupby(['store_nbr','family'])['sales'].shift(1).rolling(window).mean()

# Fill NaNs in lag/rolling features with 0 (or another strategy)
train.fillna(0, inplace=True)


In [46]:
categorical_cols = ['family','city','state','store_type','day_type','Week_day']

le_dict = {}
for col in categorical_cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])
    le_dict[col] = le  # store for inverse_transform if needed


In [47]:
target = 'sales'

features = [
    'store_nbr','onpromotion','cluster','dcoilwtico','Year','Month','Day','Week_day',
    'family','city','state','store_type','day_type'
]

# Add lag and rolling features
lag_features = [f'sales_lag_{lag}' for lag in lag_days]
rolling_features = [f'sales_roll_mean_{w}' for w in window_sizes]

features += lag_features + rolling_features


In [48]:
# Use last 3 months as validation
val_cutoff = '2017-06-01'
train_df = train[train['date'] < val_cutoff]
val_df = train[train['date'] >= val_cutoff]

X_train = train_df[features]
y_train = train_df[target]

X_val = val_df[features]
y_val = val_df[target]


In [49]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# Create DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

# Set XGBoost parameters
params = {
    'objective':'reg:squarederror',
    'eval_metric':'rmse',
    'eta':0.1,
    'max_depth':8,
    'subsample':0.8,
    'colsample_bytree':0.8,
    'seed':42
}

evallist = [(dtrain, 'train'), (dval, 'eval')]

# Train model
model = xgb.train(params, dtrain, num_boost_round=500, early_stopping_rounds=50, evals=evallist, verbose_eval=50)


[0]	train-rmse:985.54640	eval-rmse:1210.47868
[50]	train-rmse:221.26037	eval-rmse:242.48443
[100]	train-rmse:195.50430	eval-rmse:236.92067
[150]	train-rmse:180.39317	eval-rmse:233.28549
[200]	train-rmse:168.65192	eval-rmse:232.05108
[250]	train-rmse:161.03057	eval-rmse:231.31991
[281]	train-rmse:156.10489	eval-rmse:232.46703


In [50]:
# Predict on validation
y_pred = model.predict(dval)

rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f'Validation RMSE: {100*rmse/np.mean(y_val)}')


Validation RMSE: 48.34120116942862


In [51]:
from sklearn.metrics import mean_absolute_error
MAE=mean_absolute_error(y_val,y_pred)
print("MAE%",100*MAE/np.mean(y_val))

MAE% 12.834162560812535
